In [6]:
# This script is used to compare two ensemble outputs (e.g., gauge-based GMET and NLDAS-based GMET)
import matplotlib
matplotlib.use('Agg')
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from   matplotlib.dates import DateFormatter
import os
import pandas as pd
import xarray as xr
import datetime

def read_ens(out_forc_name_base, ens_num):
    for i in range(ens_num):
        ens_file = os.path.join(out_forc_name_base + '.' + str('%03d' % (i+1)) +'.nc')
        
        f=xr.open_dataset(ens_file)
        pcp = f.variables['Prcp'][:]
        t_mean = f.variables['Temp'][:]
#         t_range = f.variables['Prcp'][:]

        if i == 0:
            lats = f['lat'].values[:] #shape (y,x)
            lons = f['lon'].values[:]
            time = pd.DatetimeIndex(f['time'][:].dt.floor('H').to_pandas())
                 
            pcp_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))# create ens array 
            t_mean_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))
#             t_range_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))

        pcp_ens[:,:,:,i] = pcp
        t_mean_ens[:,:,:,i] = t_mean
#         t_range_ens[:,:,:,i] = t_range
       
    return lats, lons, time, pcp_ens, t_mean_ens#, t_range_ens

# ========================================================================================================================
root_dir='/glade/u/home/hongli/scratch/2019_10_01gssha/ens_forc_wrf2/scripts'
wrf_raw_file = os.path.join(root_dir,'step1_asc_to_nc/WestWRF_2017120200_2018040723.nc')
result_dir = os.path.join(root_dir,'step11_downscale_daily2hr')
ens_num = 10 #100

# plot_date_start = '2017-12-02 00'
# plot_date_end = '2018-04-07 23'
# plot_date_start = '2018-03-12 21'
# plot_date_end = '2018-03-16 10'
plot_date_start = '2018-01-24 21'
plot_date_end = '2018-01-25 03'

time_format = '%Y-%m-%d %H'
lb_perct = 5
ub_perct = 95

formatter = DateFormatter('%m/%d/%y') #'%Y-%m-%d'
line_marker_size = 0 #1.5 #3
line_width = 0.2
line_alpha = 0.7
scatter_marker_size = 5 #8
scatter_alpha = 0.8
bound_alpha = 0.8
dpi_value = 100

output_dir=os.path.join(root_dir,'step12_plot_temporal_ens_hour_spettigue_3mb')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# define plot mask for stn and nldas ensemble, repsectively.
plot_date_start_obj = datetime.datetime.strptime(plot_date_start, '%Y-%m-%d %H')
plot_date_end_obj = datetime.datetime.strptime(plot_date_end, '%Y-%m-%d %H')

# ========================================================================================================================
# read historical wrf data
print('Read WRF data')
f_wrf = xr.open_dataset(wrf_raw_file)
wrf_time = pd.to_datetime(f_wrf['time'].values[:]).strftime(time_format)
wrf_lats = f_wrf['lat'] #(21)
wrf_lons = f_wrf['lon'] #(18)
wrf_prcp = f_wrf['Prcp'].values[:] # (time, lat, lon). unit: mm/day
wrf_tmean = f_wrf['Temp'].values[:] # (time, lat, lon). unit: Fahrenheit

wrf_time = np.asarray([datetime.datetime.strptime(t, time_format) for t in wrf_time])

wrf_mask_time = (wrf_time>=plot_date_start_obj) & (wrf_time<=plot_date_end_obj)
prcp_mean = np.nanmean(np.nanmean(wrf_prcp[wrf_mask_time,:,:], axis = 2), axis=1) # spatial mean over all grids. shape (time). 
tmean_mean = np.nanmean(np.nanmean(wrf_tmean[wrf_mask_time,:,:], axis = 2), axis=1)

# ========================================================================================================================
print('Plot')
# loop through all uniform tests
test_folders = [d for d in os.listdir(result_dir)]
test_folders = sorted(test_folders)
for test_folder in test_folders[0:1]:
    
    print(test_folder)

    # read output ensemble
    wrf_out_forc_name_base = os.path.join(result_dir,test_folder,'ens_forc')
    lats, lons, wrf_ens_time, wrf_pcp_ens, wrf_t_mean_ens = read_ens(wrf_out_forc_name_base, ens_num)

    # define plot mask for nldas ensemble
    wrf_ens_mask_time = (wrf_ens_time>=plot_date_start_obj) & (wrf_ens_time<=plot_date_end_obj)
    
    # calculate mean and bounds
    pcp_ens  = np.nanmean(np.nanmean(wrf_pcp_ens[wrf_ens_mask_time,:,:,:], axis=2),axis=1) # spatial mean over all grids. shape (time, ens).
    pcp_ens_mean = np.nanmean(pcp_ens, axis=1) # shape (time)
#     pcp_ens_lb = np.percentile(pcp_ens, lb_perct, axis = 1) # shape (time)
#     pcp_ens_ub = np.percentile(pcp_ens, ub_perct, axis = 1)

    tmean_ens  = np.nanmean(np.nanmean(wrf_t_mean_ens[wrf_ens_mask_time,:,:,:], axis=2),axis=1) 
    tmean_ens_mean = np.nanmean(tmean_ens, axis=1) 
#     tmean_ens_lb = np.percentile(tmean_ens, lb_perct, axis = 1)
#     tmean_ens_ub = np.percentile(tmean_ens, ub_perct, axis = 1)
       
    # plot
    nrow=2 # pcp, tmean
    ncol=3 # time series, Q-Q plot
    fig = plt.figure(constrained_layout=False, figsize=(12,10*0.75))
    
    mpl.rcParams['savefig.pad_inches'] = 0 # remove any padding from the edges of the figure when saved by savefig
    #     plt.annotate('Grid ('+str(y_id)+','+str(x_id)+')',(0.05,0.95),xycoords='figure fraction',fontsize='medium',fontweight='semibold')
    
    gs = fig.add_gridspec(nrow, ncol)
    ax00 = fig.add_subplot(gs[0, 0:2])
    ax01 = fig.add_subplot(gs[0, 2])
    ax10 = fig.add_subplot(gs[1, 0:2])
    ax11 = fig.add_subplot(gs[1, 2])
    
    # WRF vs. Ensemble [PCP]
    for mb in range(ens_num):
        if mb == 0:
            ax00.plot_date(wrf_ens_time[wrf_ens_mask_time], pcp_ens[:,mb], 'b', linewidth=line_width, markersize=line_marker_size, alpha=line_alpha, label='Ens member')
        else:
            ax00.plot_date(wrf_ens_time[wrf_ens_mask_time], pcp_ens[:,mb], 'b', linewidth=line_width, markersize=line_marker_size, alpha=line_alpha)
#     ax00.fill_between(wrf_ens_time[wrf_ens_mask_time], pcp_ens_lb, pcp_ens_ub, linewidth=0, facecolor='grey', alpha=bound_alpha, label='Ens 90% unc bounds')
    ax00.plot_date(wrf_time[wrf_mask_time], prcp_mean, 'r', linewidth=1, markersize=line_marker_size, alpha=0.9, label='WRF') 
    ax01.scatter(prcp_mean, pcp_ens_mean, s=scatter_marker_size, c='k', marker='o', edgecolors='None', alpha=scatter_alpha)
    
    # WRF vs. Ensemble [TMEAN]
    for mb in range(ens_num):
        if mb == 0:
            ax10.plot_date(wrf_ens_time[wrf_ens_mask_time], tmean_ens[:,mb], 'b', linewidth=line_width, markersize=line_marker_size, alpha=line_alpha, label='Ens member')
        else:
            ax10.plot_date(wrf_ens_time[wrf_ens_mask_time], tmean_ens[:,mb], 'b', linewidth=line_width, markersize=line_marker_size, alpha=line_alpha)
#     ax10.fill_between(wrf_ens_time[wrf_ens_mask_time], tmean_ens_lb, tmean_ens_ub, linewidth=0, facecolor='grey', alpha=bound_alpha, label='Ens 90% unc bounds')
    ax10.plot_date(wrf_time[wrf_mask_time], tmean_mean, 'r', linewidth=1, markersize=line_marker_size, alpha=0.9, label='WRF') 
    ax11.scatter(tmean_mean, tmean_ens_mean, s=scatter_marker_size, c='k', marker='o', edgecolors='None', alpha=scatter_alpha)
    
    # 45 degree line in Q-Qplot
    axes = [ax01, ax11]
    for ax in axes:
        left, right = ax.get_xlim()
        bottom, top = ax.get_ylim()
        ax_min = min([left, bottom])
        ax_max = max([right, top])
        ax.set_xlim([ax_min, ax_max])
        ax.set_ylim([ax_min, ax_max])
        ax.plot([ax_min, ax_max],[ax_min, ax_max],color='grey',linewidth=1.0)
    
    # title
    ax00_title_str = '(a) Precipitation' 
    ax10_title_str = '(b) Temperature' 
    axes_title_str=[ax00_title_str, ax10_title_str]
    axes=[ax00, ax10]
    for i in range(len(axes)):
        ax=axes[i]
        title_str=axes_title_str[i]
        ax.set_title(title_str, fontsize='small', fontweight='semibold')
                
    # x-axis label and legend
    axes=[ax00, ax10]
    for ax in axes:
        ax.set_xlabel('Date', fontsize='small')
        ax.set_xlim(left=plot_date_start_obj, right=plot_date_end_obj)
        ax.xaxis.set_major_formatter(formatter)
        ax.xaxis.set_tick_params(labelsize='small')#rotation=30,
        ax.legend(loc='upper right', fontsize='small', framealpha=0.5) 
    
    # y-axis label and limit
    for ax in [ax00, ax10]:
        ax.set_ylabel('Precipitation (mm/hr)', fontsize='small')
    for ax in [ax10]:
        ax.set_ylabel('Temperature (Fahrenheit)', fontsize='small')
    
    for ax in [ax01, ax11]:
        ax.set_xlabel('WRF', fontsize='small')
        ax.set_ylabel('Ens mean', fontsize='small')
    
    plt.rc('xtick',labelsize='small')
    plt.rc('ytick',labelsize='small') 
    
    fig.tight_layout()
    output_filename = test_folder+'.png'
    fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
    plt.close(fig)
    
    del lats, lons, wrf_ens_time, wrf_pcp_ens, wrf_t_mean_ens#, wrf_t_range_ens
print('Done')

Read WRF data
Plot
046grids
Done


In [25]:
np.shape(wrf_pcp_ens),np.shape(wrf_prcp)

((3058, 22, 19, 100), (3058, 22, 19))

In [27]:
wrf_pcp_ens[8:8+24,10,10,0],wrf_prcp[8:8+24,10,10]

(array([3.97355308e-02, 3.97355308e-02, 3.97355308e-02, 3.97355308e-02,
        3.97355308e-02, 3.97355308e-02, 3.97355308e-02, 3.97355308e-02,
        3.97355308e-02, 3.97355308e-02, 3.97355308e-02, 3.97355308e-02,
        3.97355308e-02, 3.97355308e-02, 3.97355308e-02, 3.97355308e-02,
        9.70171055e-06, 1.24675150e-02, 2.49253282e-02, 3.73831413e-02,
        2.49259191e-02, 1.24686953e-02, 1.14715531e-05, 1.47751632e-05]),
 array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 2.1399585e-05, 4.2799170e-05,
        6.4198750e-05, 8.2500800e-02, 1.6493740e-01, 2.4737400e-01,
        1.6494131e-01, 8.2508610e-02, 7.5910260e-05, 9.7771110e-05]))

In [23]:
wrf_time[8:8+24],wrf_ens_time[8:8+24]

(array([datetime.datetime(2017, 12, 2, 0, 0),
        datetime.datetime(2017, 12, 2, 1, 0),
        datetime.datetime(2017, 12, 2, 2, 0),
        datetime.datetime(2017, 12, 2, 3, 0),
        datetime.datetime(2017, 12, 2, 4, 0),
        datetime.datetime(2017, 12, 2, 5, 0),
        datetime.datetime(2017, 12, 2, 6, 0),
        datetime.datetime(2017, 12, 2, 7, 0),
        datetime.datetime(2017, 12, 2, 8, 0),
        datetime.datetime(2017, 12, 2, 9, 0),
        datetime.datetime(2017, 12, 2, 10, 0),
        datetime.datetime(2017, 12, 2, 11, 0),
        datetime.datetime(2017, 12, 2, 12, 0),
        datetime.datetime(2017, 12, 2, 13, 0),
        datetime.datetime(2017, 12, 2, 14, 0),
        datetime.datetime(2017, 12, 2, 15, 0),
        datetime.datetime(2017, 12, 2, 16, 0),
        datetime.datetime(2017, 12, 2, 17, 0),
        datetime.datetime(2017, 12, 2, 18, 0),
        datetime.datetime(2017, 12, 2, 19, 0),
        datetime.datetime(2017, 12, 2, 20, 0),
        datetime.dateti

In [12]:
wrf_time

array([datetime.datetime(2017, 12, 1, 0, 0),
       datetime.datetime(2017, 12, 1, 0, 0),
       datetime.datetime(2017, 12, 1, 0, 0), ...,
       datetime.datetime(2018, 4, 8, 0, 0),
       datetime.datetime(2018, 4, 8, 0, 0),
       datetime.datetime(2018, 4, 8, 0, 0)], dtype=object)